In [12]:

# TODO: for each ds, load it, filter relevant columns and join on nft_address

In [2]:
import os
import sqlite3

import matplotlib.pyplot as plt
import numpy as np
import nfts.dataset

import pandas as pd

In [6]:
dirname = os.path.dirname(os.path.abspath("__file__"))
DATASET_PATH = os.path.join(dirname, "nfts.sqlite")
ds = nfts.dataset.FromSQLite(DATASET_PATH)

nfts_df = ds.load_dataframe("nfts")
nfts_df.head()

,address,name,symbol
0,0xF4ee95274741437636e748DdAc70818B4ED7d043,The Doge Pound,DOGGY
1,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC
2,0x7403AC30DE7309a0bF019cdA8EeC034a5507cbB3,Characters,CHAR
3,0x511372B44231a31527025a3D273C1dc0a83D77aF,Maps Restored,MAP
4,0x2E956Ed3D7337F4Ed4316A6e8F2EdF74BF84bb54,Neon Junkies,JUNKIES


Collections with most contracts

In [11]:
collections_df = nfts_df.groupby('name', as_index=False).size().rename(columns={'size':'num_addresses'})
collections_df.sort_values('num_addresses', ascending=False, inplace=True)

collections_df.head()

,name,num_addresses
4894,NFT Basket,68
0,,19
4933,NFT-LOTTERY,16
7406,Tinlake Loan Token,13
568,Artiikorea,9


# Tables Joining

- nfts
    - address
- mints
    - transaction_value
    - timestamp
    - block_number ???
    - to_address ???
- transfers
    - transaction_value
    - block_number ???
- current_owners
    - owner
- current_market_values
    - market_value
- transfer_statistics_by_address
    - number of transfers

Tables loading and projection

In [13]:
# nfts_df = nfts_df[['nft_address']] It's not needed, unless for graphical representation including name of collections

mints_df = ds.load_dataframe("mints")
mints_df = mints_df[['nft_address', 'transaction_value', 'timestamp']]

transfers_df = ds.load_dataframe("transfers")
transfers_df = transfers_df[['nft_address', 'transaction_value']]
# TODO: aggregation functions for nft_address (number of rows should be equal to mints !!!)
# TODO: for each nft_address, number of transfers (see transfers.ipynb)

current_owners_df = ds.load_dataframe("current_owners")
current_owners_df = current_owners_df[['nft_address', 'owner']]

current_market_values_df = ds.load_dataframe("current_market_values")
current_market_values_df = current_market_values_df[['nft_address', 'market_value']]


Join in one table, by nft_address

In [ ]:
nfts_clustering_df = mints_df.merge('', on='')

# Clustering

# Conclusions